In [ ]:
import mbuild as mb
import matplotlib.pyplot as plt
import numpy as np
from foyer import Forcefield
from calc_numberdensity import calc_number_density

In [ ]:
water = mb.load('files/tip3p.mol2')
water.name = 'SOL'

In [ ]:
c_ff = Forcefield('files/carbon.xml')
spce = Forcefield('files/spce.xml')

In [ ]:
system = mb.recipes.GraphenePoreSolvent(pore_depth=2,side_dim=2,
                    n_sheets=3,pore_width=1.2,x_bulk=1.5,
                    solvent=water,n_solvent=300)
box = mb.Box(system.periodicity)

In [ ]:
water = mb.Compound()
graphene = mb.Compound()
for child in system.children:
    if child.name == 'SOL':
        water.add(mb.clone(child))
    else:
        graphene.add(mb.clone(child))

water_pmd = spce.apply(water, residues='SOL')           
pore_pmd = c_ff.apply(graphene)

In [ ]:
system = water_pmd + pore_pmd
system.box[:3] = box.maxs * 10.0
system.save('init.gro', overwrite=True, combine='all')
system.save('init.top', overwrite=True, combine='all')

In [ ]:
!gmx grompp -f files/nvt.mdp -c init.gro -p init.top -o nvt.tpr

In [ ]:
!gmx mdrun -v -deffnm nvt

In [ ]:
reference_num_den = np.loadtxt('reference-data/numden.txt')

In [ ]:
gro_file = 'nvt.gro'
trj_file = 'nvt.trr'
top_file = 'init.top'

In [ ]:
box_range = [0.84, 2.04]
mins = [3.61, 0, 0]
maxs = [5.76, 0, 0]
area = 6.45

In [ ]:
plt.figure()
plt.xlim(box_range[0], box_range[1])
rho, bins = calc_number_density(gro_file, trj_file, top_file, area, dim=1,
                                    box_range=box_range, n_bins=100, frame_range=range(0,11),
                                    maxs=maxs, mins=mins)
plt.plot(bins, rho[0], label='Water: Frames {}-{}'.format(0,11))
plt.plot(reference_num_den[:,0], reference_num_den[:,1], label='Reference Water: Frames {}-{}'.format(0,11))
plt.legend()
plt.savefig('number-density.pdf')